### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [361]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [362]:
clf = DecisionTreeClassifier()

In [363]:
a = np.array([1,2,3,1,1,3])
b = np.array([1,2,3,4,5,3,3,3,4,4,5,1,1,21,1])
a = np.array(a)
b = np.array(b)

np.concatenate((a, b), axis=0)

array([ 1,  2,  3,  1,  1,  3,  1,  2,  3,  4,  5,  3,  3,  3,  4,  4,  5,
        1,  1, 21,  1])

In [364]:
d = np.array(0.344)

In [365]:
d.shape

()

In [366]:
feature_ids = [i for i in range(10)]
np.random.shuffle(feature_ids)

In [367]:
np.unique(a,return_counts=True)[1] / a.shape[0]


array([0.5       , 0.16666667, 0.33333333])

In [33]:
a = np.array([1,2,3])

In [34]:
b = np.array([[2,3,4], [3,4,5]])

In [90]:
1 - a**2

array([ 0, -3, -8])

In [379]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.depth = 0
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise
    
    
    def __func_gini(self, p):
        return 1 - (p ** 2).sum(axis=0)
        
        
    def __gini(self, matr_left, N_l, matr_right, N_r, y_p):
        
        N_m = N_l + N_r
        
        return (self.__func_gini(y_p)
                - N_l / N_m * self.__func_gini(matr_left)
                - N_r / N_m * self.__func_gini(matr_right))
    
    
    
    def __func_entropy(self, y_in_v):
        
        N = y_in_v.shape[0]
        
        p = np.unique(y_in_v, return_counts=True)[1] / N
        return -(p * np.log2(p)).sum()
        
    def __entropy(self, y_left, y_right):
        if y_left.shape[0] == 0 or y_right.shape[0] == 0: 
            return 10
        y_left = y_left.astype('float')
        y_right = y_right.astype('float')
        N_l = y_left.shape[0]
        N_r  = y_right.shape[0]
        N_m = N_l + N_r

        return (self.__func_entropy(np.concatenate((y_left, y_right), axis=0))
                - N_l / N_m * self.__func_entropy(y_left)
                - N_r / N_m * self.__func_entropy(y_right))

    
    
    
    def __func_misclass(self, y_in_v):
        classes, classes_cnt = np.unique(y_in_v, return_counts=True)
        k = classes[classes_cnt.argmax()]
        return 1. / y_in_v.shape[0] * (y_in_v == k).sum()
    
    def __misclass(self, y_left, y_right):
        
        if y_left.shape[0] == 0 or y_right.shape[0] == 0: 
            return 10
        y_left = y_left.astype('float')
        y_right = y_right.astype('float')
        N_l = y_left.shape[0]
        N_r  = y_right.shape[0]
        N_m = N_l + N_r
        return (self.__func_misclass(np.concatenate((y_left, y_right), axis=0))
                - N_l / N_m * self.__func_misclass(y_left)
                - N_r / N_m * self.__func_misclass(y_right))

    
    
    
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(n_feature))])
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        ''' информативносьть  + Возвращает порог'''
        
        x_sort, y_sort = self.__sort_samples(x, y)
        all_classes_in_y = np.bincount(y)
        ln = all_classes_in_y.shape[0]
        matr_right = []
        for i in range(1, len(y) - 1):
            threshold = x_sort[i]
            matr_right.append(np.bincount(y_sort[:i], minlength=ln))
        matr_right = np.array(matr_right)
        
        
#         print(np.bincount(y_sort[:1], minlength=ln))    
#         print(matr_right.shape,all_classes_in_y.shape)
        #print(matr_right.shape, all)
        matr_left = all_classes_in_y - matr_right
        
        matr_left = matr_left.astype(float)
        matr_right = matr_right.astype(float)
        N_l = np.sum(matr_left)
        N_r = np.sum(matr_right)
        matr_left /= ln
        matr_right /= ln
        matr_left = matr_left.T
        matr_right = matr_right.T
        gains = self.G_function(matr_left, N_l, matr_right, N_r, all_classes_in_y/ln)
        #print(gains)
        ind = gains.argmin()
            
        
        return gains[ind], (x_sort[ind + 1] + x_sort[ind])/2.
    
    
    

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        
        self.depth = max(self.depth, depth)
        
        
        if ((self.max_depth is not None and self.max_depth < depth)
                or (y.shape[0] < self.min_samples_split)):
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return
        if y.shape[0] == 2:
            if y[0] == y[1]:
                self.tree[node_id] = (self.LEAF_TYPE, y[0],
                                  y[0])
                return
            else:
                self.tree[node_id] = (self.NON_LEAF_TYPE, 0,
                              (x[0,0] + x[1,0])/2.)
                self.__fit_node(np.array(x[0]), np.array([y[0]]), 2*node_id + 1, depth + 1, pred_f)
                self.__fit_node(np.array(x[1]), np.array([y[1]]), 2*node_id + 2, depth + 1, pred_f)
                return
                
        feature_ids = self.get_feature_ids(x.shape[1])
        threshold = np.array([self.__find_threshold(x[:, feature_id], y)
                             for feature_id in feature_ids])
        #print(threshold)
        best_split_id = np.argmin(threshold[:, 0])
        gs, best_split = threshold[best_split_id]

        if best_split is None:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return

        x_l, x_r, y_l, y_r = \
            self.__div_samples(x, y, feature_ids[best_split_id], best_split)
        if x_l.shape[0] == 0 or x_r.shape[0] == 0:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return

        self.tree[node_id] = (self.NON_LEAF_TYPE, feature_ids[best_split_id],
                              best_split)

        

        self.__fit_node(x_l, y_l, 2*node_id + 1, depth + 1, pred_f)
        self.__fit_node(x_r, y_r, 2*node_id + 2, depth + 1, pred_f)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= y.size

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [380]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, criterion="gini")
clf = DecisionTreeClassifier(min_samples_split=3, criterion="gini")

In [381]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [382]:
%time clf.fit(X_train, y_train)

CPU times: user 1.97 ms, sys: 0 ns, total: 1.97 ms
Wall time: 1.13 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [383]:
clf.get_depth()

6

In [384]:
%time my_clf.fit(X_train, y_train)

CPU times: user 79.3 ms, sys: 63 µs, total: 79.4 ms
Wall time: 78.6 ms


In [385]:
my_clf.depth

10

## Проверка качества работы на wine

In [386]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

1.0

In [387]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.3656565656565656

## Подготовка данных Speed Dating Data 

## Проверка скорости работы на Speed Dating Data 

In [ ]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

In [ ]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

## Проверка качества работы на Speed Dating Data

In [ ]:
# тут должен быть код типа %time clf.fit(X_train, y_train)

In [ ]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)

## Задание 3

In [ ]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini',
                 max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - ((l_c ** 2 / l_s).sum(axis=1)
                    + (r_c ** 2 / r_s).sum(axis=1)) / (l_s + r_s)[0]

    def __simple_gini(self, y):
        _, counts = np.unique(y, return_counts=True)
        return y.size * (1 - np.sum((counts / y.size) ** 2))

    def __entropy(self, l_c, l_s, r_c, r_s):
        return - (((l_c / l_s).log2() * l_c).sum(axis=1)
                  + ((r_c / r_s).log2() * r_c).sum(axis=1)) / (l_s + r_s)[0]

    def __simple_entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        return - y.size * (np.sum((counts / y.size)
                                  * np.log2(counts / y.size)))

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (l_c.max(axis=1) + r_c.max(axis=1)) / (l_s + r_s)[0]

    def __simple_misclass(self, y):
        _, counts = np.unique(y, return_counts=True)
        return y.size * (1 - np.max(counts / y.size))

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(n_feature))])

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода? - сортирует x и y по
        # признакам и получает количество классов
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # Что делает этот блок кода? - находит индексы
        # в которых происходит смена y, в случае если таких нет
        # возвращает None если y не меняется
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)

        if len(r_border_ids) == 0:
            return np.inf, None

        # Что делает этот блок кода? - получает количество одинаковых
        # элементов в интревале, потом заполняем единицами места где происходит
        # смена. Затем делаем матрицу количества классов для каждого интервала
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        # Что делает этот блок кода? - считаем количество классов
        # в левой и правой части разбиения
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода? - считаем неопределенность и
        # минимизируем её
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода? - возвращает значение наименьшей
        # неопределенности и порога для признака
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        if ((self.max_depth is not None and self.max_depth < depth)
                or (y.shape[0] < self.min_samples_split)):
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return

        feature_ids = self.get_feature_ids(x.shape[1])
        threshold = np.array([self.__find_threshold(x[:, feature_id], y)
                             for feature_id in feature_ids])
        best_split_id = np.argmin(threshold[:, 0])
        gs, best_split = threshold[best_split_id]

        if best_split is None:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return

        x_l, x_r, y_l, y_r = \
            self.__div_samples(x, y, feature_ids[best_split_id], best_split)
        if x_l.shape[0] == 0 or x_r.shape[0] == 0:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return

        self.tree[node_id] = (self.NON_LEAF_TYPE, feature_ids[best_split_id],
                              best_split)

        if self.G_function == self.__gini:
            gain_fun = self.__simple_gini
        elif self.G_function == self.__entropy:
            gain_fun = self.__simple_entropy
        elif self.G_function == self.__misclass:
            gain_fun = self.__simple_misclass

        self.feature_importances_[best_split_id] += \
            gain_fun(y) - gain_fun(y_l) - gain_fun(y_r)

        self.__fit_node(x_l, y_l, 2*node_id + 1, depth + 1, pred_f)
        self.__fit_node(x_r, y_r, 2*node_id + 2, depth + 1, pred_f)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= y.size

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

## Задание 4

In [888]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.depth = 0
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise
    
    
    def __func_gini(self, y_in_v):
        N = y_in_v.shape[0]
        p = np.unique(y_in_v, return_counts=True)[1] / N
        return 1 - (p ** 2).sum()
        
        
    def __gini(self, y_left, y_right):
        if y_left.shape[0] == 0 or y_right.shape[0] == 0: 
            return 10
        y_left = y_left.astype('float')
        y_right = y_right.astype('float')
        N_l = y_left.shape[0]
        N_r  = y_right.shape[0]
        N_m = N_l + N_r
        return (self.__func_gini(np.concatenate((y_left, y_right), axis=0))
                - N_l / N_m * self.__func_gini(y_left)
                - N_r / N_m * self.__func_gini(y_right))
    
    
    
    def __func_entropy(self, y_in_v):
        
        N = y_in_v.shape[0]
        
        p = np.unique(y_in_v, return_counts=True)[1] / N
        return -(p * np.log2(p)).sum()
        
    def __entropy(self, y_left, y_right):
        if y_left.shape[0] == 0 or y_right.shape[0] == 0: 
            return 10
        y_left = y_left.astype('float')
        y_right = y_right.astype('float')
        N_l = y_left.shape[0]
        N_r  = y_right.shape[0]
        N_m = N_l + N_r

        return (self.__func_entropy(np.concatenate((y_left, y_right), axis=0))
                - N_l / N_m * self.__func_entropy(y_left)
                - N_r / N_m * self.__func_entropy(y_right))

    
    
    
    def __func_misclass(self, y_in_v):
        classes, classes_cnt = np.unique(y_in_v, return_counts=True)
        k = classes[classes_cnt.argmax()]
        return 1. / y_in_v.shape[0] * (y_in_v == k).sum()
    
    def __misclass(self, y_left, y_right):
        
        if y_left.shape[0] == 0 or y_right.shape[0] == 0: 
            return 10
        y_left = y_left.astype('float')
        y_right = y_right.astype('float')
        N_l = y_left.shape[0]
        N_r  = y_right.shape[0]
        N_m = N_l + N_r
        return (self.__func_misclass(np.concatenate((y_left, y_right), axis=0))
                - N_l / N_m * self.__func_misclass(y_left)
                - N_r / N_m * self.__func_misclass(y_right))

    
    
    
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(n_feature))])
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        x_sort,y_sort = self.__sort_samples(x, y)
        ln =  x.shape[0]
        step = (x.max() - x.min()) / ln
        
        if y.shape[0] == 2:
            gains = np.array([self.G_function(y_sort[x_sort <= x_sort[0] + 1e-16], y_sort[x_sort > x_sort[0] + 1e-16])])
        else:
            gains = np.array([self.G_function(y_sort[x_sort <= x_sort[i] + 1e-16], y_sort[x_sort > x_sort[i] + 1e-16])
                              for i in range(0, ln - 1)])
        
        if gains.min() == 10:
            return np.inf, None
        
        ind = gains.argmin()
        
        return gains[ind], (x_sort[ind])

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        #print(x.shape)
        self.depth = max(self.depth, depth)
        if ((self.max_depth is not None and self.max_depth < depth)
                or (y.shape[0] < self.min_samples_split)):
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return

        feature_ids = self.get_feature_ids(x.shape[1])
        threshold = np.array([self.__find_threshold(x[:, feature_id], y)
                             for feature_id in feature_ids])
        
        best_split_id = np.argmin(threshold[:, 0])
        gs, best_split = threshold[best_split_id]

        if best_split is None:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return

        x_l, x_r, y_l, y_r = \
            self.__div_samples(x, y, feature_ids[best_split_id], best_split)
        if x_l.shape[0] == 0 or x_r.shape[0] == 0:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return

        self.tree[node_id] = (self.NON_LEAF_TYPE, feature_ids[best_split_id],
                              best_split)

        

        self.__fit_node(x_l, y_l, 2*node_id + 1, depth + 1, pred_f)
        self.__fit_node(x_r, y_r, 2*node_id + 2, depth + 1, pred_f)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= y.size

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

## Задание 5